# Day 2 Goals
- Run alignment job for at least 1 sample
- Load counts/TPM matrices into R and practice matrix manipulation, subsetting, etc. 
- Practice making plots with `ggplot2`
- Identify differentially expressed genes in a dataset

# Bulk RNA-seq analysis

Yesterday we got started working on HPC, learning Unix commands, and running bash scripts. That is a lot to do in just a few hours, so we expect that today we will also be working on navigating the command line and aligning reads. We can answer questions and troubleshoot altogether if many people have similar questions, and/or some of us can break out into small groups to address specific questions.

There are a few more steps post-alignment we need to do in order to get our gene-by-sample counts matrices. We want to make sure to touch on everyone's interests as much as possible (and give you the tools needed to start thinking about your projects!), so we have pre-made counts matrices so we can get started on secondary analysis and plotting.

### Log in
https://hpc3.rcic.uci.edu/biojhub3/ 

## Option 1: Download pre-made counts matrix and TPM matrix

If you need them, copy the counts and TPM matrices from my public directory to your home directory.

```
cp /pub/erebboah/cosmos/FSHD_bulkRNA/kallisto_counts/fshd.counts.matrix.csv .
cp /pub/erebboah/cosmos/FSHD_bulkRNA/kallisto_counts/fshd.tpm.matrix.csv .
```

## Option 2: Build matrices from kallisto output
Fairlie can teach you how to merge all the individual abundance matrices together and convert the transcripts to genes.

# Analyze data in R

In [ ]:

tpm = as.matrix(read.csv("fshd.tpm.matrix.csv", row.names = 1))
counts = as.matrix(read.csv("fshd.counts.matrix.csv", row.names = 1))

In [ ]:
head(tpm)
head(counts)
dim(tpm)

We have 59,429 genes by 8 samples. The python script outputs both the Ensembl gene ID and the gene name separated by "|"; let's grab the gene name to be more human readable.

In [ ]:
library(stringr)

In [ ]:
gene_name = sapply(strsplit(rownames(tpm), "[|]"), "[[", 2) # string manipulation using | as a separator
head(gene_name)

In [ ]:
rownames(tpm) = gene_name
head(tpm) # much better
rownames(counts) = gene_name
head(counts) 

59,429 genes sounds like a lot-- probably because many aren't expressed, such as predicted genes. We can filter out rows containing all 0 values using the `rowSums()` function.

In [ ]:
### Filter out genes with 0 count
tpm = tpm[rowSums(tpm[])>0,]
dim(tpm)

### PCA and outlier removal
We'll use the TPM matrix to perform principal component analysis ([PCA](https://builtin.com/data-science/step-step-explanation-principal-component-analysis)).

In [ ]:
# Run PCA algorithm and plot
pca <- prcomp(t(log2(tpm+1)))
# Plot PCA1 and PCA2
plot(pca$x[,1], pca$x[,2])

We can make a fancier PCA plot with `ggplot`.

In [ ]:
# Grab x from pca output
pca_out <- as.data.frame(pca$x)

# Get percent variance explained for x and y axes
percentage <- round(pca$sdev/sum(pca$sdev) * 100, 2)
percentage <- paste0(colnames(pca_out), " (", paste0(as.character(percentage), "%", ")"))

In [ ]:
# Use column names and string manipulation to make metadata
pca_out$Sample <- colnames(counts)
pca_out$Genotype <- sapply(strsplit(colnames(counts), "_"), "[[", 1)
pca_out$Timepoint <- sapply(strsplit(colnames(counts), "_"), "[[", 4)
pca_out$Replicate <- sapply(strsplit(colnames(counts), "_"), "[[", 5)

In [ ]:
library(ggplot2)
library(ggrepel)
library(factoextra)
library(tidyverse)

# Labels are sample, colors are timepoint
p = ggplot(pca_out,aes(x=PC1,y=PC2,color=Timepoint))+
  geom_point(aes(shape=Genotype),size=1) + 
  xlab(percentage[1]) + ylab(percentage[2]) +
  geom_label_repel(aes(label = Sample),
                   box.padding   = 0.35, 
                   point.padding = 0.5,
                   segment.color = 'grey50',
                   show.legend=FALSE) +
  theme_bw()
print(p)

Looks like PC1 primarily separates day 0 and day 3 samples and explains slightly more variance than PC2, which separates control and FSHD2. Let's make another PCA plot but change the [colors](https://colorbrewer2.org/) and size of the points:

In [ ]:
p = ggplot(pca_out,aes(x=PC1,y=PC2,color=Timepoint))+
  geom_point(aes(shape=Genotype),size=4) + 
  xlab(percentage[1]) + ylab(percentage[2]) +
  geom_label_repel(aes(label = Sample), 
                   box.padding   = 0.35, 
                   point.padding = 0.5,
                   segment.color = 'grey50',
                   show.legend=FALSE) +
  theme_bw() + scale_color_manual(values=c("#1b9e77", "#d95f02"))

In [ ]:
print(p)

Save figure to a PDF! Make a figures directory if you haven't done so:

In [ ]:
ggsave(file = "PCA_bulkRNA_fshd.pdf",
    width = 5.5, 
    height = 5)
dev.off()

In [ ]:
res.var <- get_pca_var(pca)
head(res.var$contrib)

In [ ]:
#### Function to extract Dimension
PCA_extract <- function(y){
res.var <- get_pca_var(pca)
res.var <- res.var$contrib
res.var <- res.var[order(-res.var[,y]),]
PCA <- res.var %>%
  as.data.frame %>% 
  head(50, by = y) 
PCA$gene_clean <-rownames(PCA)
}

In [ ]:
## Extract Dimension
PCA_1 <- PCA_extract("Dim.1")
PCA_2 <- PCA_extract("Dim.2")
head(PCA_1,50)

In [ ]:
head(PCA_2,50)

# Excercise 1

### 1) Plot PCA3 and PCA4 

### 2) Extract genes for PCA3 and PCA4


### Run differential expression analysis

In [ ]:
library(edgeR)

In [ ]:
# We need a metadata file -- basically already made it earlier with stringsplit operations
meta = pca_out[,c("Sample","Timepoint","Genotype","Replicate")]
head(meta)

#### Set up EdgeR function

In [ ]:
### Make groups
group <- factor(str_sub(meta$Sample, end=-6))
z <- tpm
### read count matrix into DGEList
y=DGEList(counts=z, group=group)
design <- model.matrix(~0+group, data=y$samples)
colnames(design) <- levels(y$samples$group)
head(design)

In [ ]:
### Calculation
y=estimateCommonDisp(y)

y=estimateTagwiseDisp(y)

fit <- glmQLFit(y, design)

In [ ]:
EdgeR_func_GLM <- function(w){

Contrast <- makeContrasts(contrasts="Control_78_Day_0-FSHD2_19_Day_0", levels=design)
qlf <- glmQLFTest(fit, contrast=Contrast)
result <- topTags(qlf,n=10000000) 
table <- as.data.frame(result)
### Select cut off
top_up <- subset(as.data.frame(result), PValue < 0.01 & logFC > 1.5)
top_genes_up <- as.data.frame(subset(tpm, row.names(tpm) %in% row.names(top_up)))

top_down <- subset(as.data.frame(result), PValue < 0.01 & logFC < -1.5)
top_genes_down <- as.data.frame(subset(tpm, row.names(tpm) %in% row.names(top_down)))
### Up Gene
top_genes_up$gene_name <- rownames(top_genes_up)
top_up$gene_name <- rownames(top_up)
### Down Gene
top_genes_down$gene_name <- rownames(top_genes_down) 
top_down$gene_name <- rownames(top_down)
### Make table
top_up <-left_join(top_up, dplyr::select(top_genes_up,gene_name), by = 'gene_name')
top_down <-left_join(top_down, dplyr::select(top_genes_down,gene_name), by = 'gene_name')
output<-list("Up"= top_up, "Down" = top_down,"table"=table)
return(output)
}

In [ ]:
#### Make comparison
#### Compare Control vs FSHD
result <- EdgeR_func_GLM("Control_78_Day_3-Control_78_Day_0")
Control_3_up <- result$Up$gene_name
Control_3_down <- result$Down$gene_name
Control_3_table <- result$table

In [ ]:
head(Control_3_table,50)

# Excercise 2

### Compare FSHD2 to Control Day 0

# Visualize Data

In [ ]:
## Load package
library(ggplot2)

### Make a volcano plot

#### Make table for plotting

In [ ]:
plot_table <- Control_3_table %>% 
  mutate(Categories = ifelse(logFC >= 1.0 & PValue <  0.01,"High in FSHD", 
    ifelse(logFC<= -1.0 & PValue <  0.01 , "Low in FSHD", "No Changes")))
head(plot_table,50)

#### Plot

In [ ]:
ggplot(plot_table, aes(x=logFC, y= -log10(PValue))) +
  geom_point(aes(colour = Categories), size=1) +
  geom_hline(yintercept=2, linetype="dashed", 
             color = "red", size=0.5) +
  geom_vline(xintercept=1, color = "blue", size=0.5) +
  geom_vline(xintercept=-1, color = "blue", size=0.5) 
  

#### Add titles and labels

In [ ]:
ggplot(plot_table, aes(x=logFC, y= -log10(PValue))) +
  geom_point(aes(colour = Categories), size=1) +
  geom_hline(yintercept=2, linetype="dashed", 
             color = "red", size=0.5) +
  geom_vline(xintercept=1, color = "blue", size=0.5) +
  geom_vline(xintercept=-1, color = "blue", size=0.5) +
  ggtitle("FSHD vs Control") +
  xlab("log2 Fold Change") + ylab("log10 Pvalue")

#### Add theme to make the plot looks nicer

In [ ]:
ggplot(plot_table, aes(x=logFC, y= -log10(PValue))) +
  geom_point(aes(colour = Categories), size=1) +
  geom_hline(yintercept=2, linetype="dashed", 
             color = "red", size=0.5) +
  geom_vline(xintercept=1, color = "blue", size=0.5) +
  geom_vline(xintercept=-1, color = "blue", size=0.5) +
  ggtitle("FSHD vs Control") +
  xlab("log2 Fold Change") + ylab("log10 Pvalue") +
  theme_bw(base_size = 20) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),legend.title = element_text(size=35),
  panel.background = element_blank(), axis.line = element_line(colour = "black"))

#### Add manual colors

In [ ]:
ggplot(plot_table, aes(x=logFC, y= -log10(PValue))) +
  geom_point(aes(colour = Categories), size=1) +
  geom_hline(yintercept=2, linetype="dashed", 
             color = "red", size=0.5) +
  geom_vline(xintercept=1, color = "blue", size=0.5) +
  geom_vline(xintercept=-1, color = "blue", size=0.5) +
  ggtitle("FSHD vs Control") +
  xlab("log2 Fold Change") + ylab("log10 Pvalue") +
  theme_bw(base_size = 20) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),legend.title = element_text(size=35),
  panel.background = element_blank(), axis.line = element_line(colour = "black")) +
  scale_colour_manual(values = c("High in FSHD"= "darkcyan", "Low in FSHD"="darkred","No Changes"= "grey")) 

#### Label genes of interest 

In [ ]:
### Store genes in a vector for labeing
vc_labels <- head(Control_3_up,10)
plot_table$genes <- rownames(plot_table)
### Plot
ggplot(plot_table, aes(x=logFC, y= -log10(PValue),label = genes)) +
  geom_point(aes(colour = Categories), size=1) +
  geom_hline(yintercept=2, linetype="dashed", 
             color = "red", size=0.5) +
  geom_vline(xintercept=1, color = "blue", size=0.5) +
  geom_vline(xintercept=-1, color = "blue", size=0.5) +
  ggtitle("FSHD vs Control") +
  xlab("log2 Fold Change") + ylab("log10 Pvalue") +
  theme_bw(base_size = 20) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),legend.title = element_text(size=35),
  panel.background = element_blank(), axis.line = element_line(colour = "black")) +
  scale_colour_manual(values = c("High in FSHD"= "darkcyan", "Low in FSHD"="darkred","No Changes"= "grey")) +
  geom_label_repel(data= subset(plot_table, genes %in% vc_labels),segment.color = "blue",
                    direction     = "both", size = 3,
                   box.padding   = 0.1,force  = 4,
                   point.padding = 0.1,max.time=10,max.overlaps = Inf)

In [ ]:
ggsave(file = "Volcano_Control_3_up.pdf",
    width = 10, 
    height = 7)
dev.off()

In [ ]:
head(subset(plot_table, genes %in% vc_labels),50)

In [ ]:
head(rownames(tpm))

In [ ]:
# Write the DE gene lists 

write.table(Control_3_table, file="Control_table_Day3",sep='\t')

# Exercise 3

### Plot volcano plot for genes upregulated in FSHD2 Day3 compared to Day0

### We use [enrichR](https://cran.r-project.org/web/packages/enrichR/vignettes/enrichR.html), which also has a [web tool](https://maayanlab.cloud/Enrichr/).

In [ ]:
library(enrichR)

In [ ]:
dbs <- c("GO_Molecular_Function_2021", "GO_Cellular_Component_2021", "GO_Biological_Process_2021")
enriched <- enrichr(Control_3_up, dbs)

In [ ]:
head(enriched[["GO_Biological_Process_2021"]])

In [ ]:
plotEnrich(enriched[[3]], showTerms = 20, numChar = 40, y = "Count", orderBy = "P.value")



# Homework
- Get comfortable in Jupyter notebook hub
- Make at least one plot using ggplot in Jupyter R notebook (check tutorial links below)
- Think about some applications of differential gene expression analysis

# Useful links
- [Intro to Jupyter notebooks](https://towardsdatascience.com/a-beginners-tutorial-to-jupyter-notebooks-1b2f8705888a)
- [ggplot tutorial](http://r-statistics.co/Complete-Ggplot2-Tutorial-Part1-With-R-Code.html)  
- [Another ggplot tutorial](https://www.publichealth.columbia.edu/sites/default/files/media/fdawg_ggplot2.html)
- [edgeR tutorial](https://web.stanford.edu/class/bios221/labs/rnaseq/lab_4_rnaseq.html)